In [1]:
#Notebook for development of the game
# Imports main modules - Player, MapGenerator, etc.
# Implements pygame controls and GUI

In [1]:
import sys, random, os, math, time, itertools
import numpy as np

#where non-dev code is located
sys.path.append('../app')

In [2]:
#install pygame
#!{sys.executable} -m pip install pygame

In [3]:
import pygame

#config stores constants and default settings
from config import Config
from map_generator import MapGenerator
from the_map import TheMap
from player import HumanPlayer, AgentPlayer, GreedyGoalAgentPlayer
from capture_the_flag import CaptureTheFlag

#helpful pygame wrapper
import pygame_utils as pyg

pygame 2.0.1 (SDL 2.0.14, Python 3.9.1)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [4]:
#MAKE SURE any changes here are reflected in app/the_map.py

class TheMap():
    def __init__(self, config, new_map_seed=0):
        self.config = config
        self.border_size = self.config.map_border_size
        self.tile_size = self.config.terrain_tile_size
        
        #make the map and get the terrain speeds associated with the map
        if new_map_seed:
            if self.config.verbose:
                print('Creating new map')
            tile_cols, tile_rows = 40, 30
            pixel_dims = (self.tile_size * tile_cols, self.tile_size * tile_rows)
            self.tile_speeds, self.map_path = MapGenerator(self.config, seed=new_map_seed).generate_map(pixel_dims, 
                                                                        save_path = self.config.maps_path)
        else:
            if self.config.verbose:
                print('Using default map')
            self.map_path = self.config.map_default_path
            self.tile_speeds = np.load(self.config.map_default_speed_array)
            
        self.middle_tile = self.tile_speeds.shape[1]//2
            
        self.blue_flag_xy = (0,0)
        self.blue_flag_area_tiles = []
        self.red_flag_xy = (0,0)
        self.red_flag_area_tiles = []
        
        self.blue_flag_in_play = False
        self.red_flag_in_play = False
        
        #store [team][player_idx]:{'xy':(x,y), 'has_flag':, 'is_incapacitated':, 'in_enemy_territory'}
        self.agent_info = {'blue':{}, 'red':{}}
        
        
    def set_flag_location(self, team, flag_x, flag_y):
        if team=='blue':
            self.blue_flag_xy = (flag_x, flag_y)
        else:
            self.red_flag_xy = (flag_x, flag_y)
        
        #get tiles in flag area
        flag_area_border_tiles = ((self.config.flag_area_size//self.tile_size) // 2) - 1
        flag_c, flag_r = self.xy_to_cr(flag_x, flag_y)
        for r in range(flag_r - flag_area_border_tiles, flag_r + flag_area_border_tiles + 1):
            for c in range(flag_c - flag_area_border_tiles, flag_c + flag_area_border_tiles + 1):
                if team=='blue':
                    self.blue_flag_area_tiles.append((r, c))
                else:
                    self.red_flag_area_tiles.append((r, c))
                    
        #if self.config.verbose:
        #    print('%s flag is on %s, %s, in area %s' % (team, flag_r, flag_c, 
        #            str(self.blue_flag_area_tiles) if team=='blue' else str(self.red_flag_area_tiles)))
                    
        
    def get_not_allowed_tiles(self):
        idx = np.where(self.tile_speeds==0)
        
        return list(zip(idx[0].tolist(), idx[1].tolist()))
    
    
    def xy_to_cr(self, x, y):
        return x // self.config.terrain_tile_size, y // self.config.terrain_tile_size
        
        
    def get_speed(self, tile_col, tile_row):
        return int(self.tile_speeds[tile_row, tile_col])
    
    
    def in_enemy_territory(self, team, tile_col):
        if team=='blue':
            return tile_col > self.middle_tile
        else:
            return tile_col < self.middle_tile
        
        
    def in_flag_area(self, team, tile_col, tile_row):
        if team=='blue':
            return (tile_row, tile_col) in self.blue_flag_area_tiles
        else:
            return (tile_row, tile_col) in self.red_flag_area_tiles


In [5]:
class Policy:
    def __init__(self, q_matrix_path=''):
        self.epsilon = 1e-10
        
        self.high_level_actions = [
            'wait', #if stunned
            'random', #a valid strategy
           'go_opponent_flag',  #go get the flag
           'go_team_flag_area',  #especially if you have the flag
           'go_opponent_flag_carrier', #tag flag carrier
           'go_nearest_opponent', #chase
           'go_nearest_teammate', #a good choice if you have the flag
           'go_nearest_incapacitated_teammate', #revive them
           'gaurd_nearest_teammate',  #get between nearest teammate and enemy
           'gaurd_teammate_flag_carrier', #get between flag carrying teammate and enemy
           'gaurd_team_flag_area', #get between flag area and enemy
           'guard_opponent_flag_area',  #especially if opponent is running the flag
           'run_away_from_nearest_opponent',  #if being chased
           'run_away_from_opponents_centroid' #allows skirting dangerous areas
          ]
        
        self.high_level_states = ['opponents_flag_in_play', 
                                  'team_flag_in_play', 
                                  'self_incapacitated', 
                                  'self_has_flag', 
                                  'self_in_enemy_territory',
                                  'nearest_teammate_has_flag',
                                  'nearest_teammate_incapacitated', 
                                  'nearest_opponent_has_flag',
                                  'nearest_opponent_incapacitated'
                                 ]
        
        self.high_level_state_codes = [tuple([int(s) for s in seq]) for seq in itertools.product('01', repeat=len(self.high_level_states))]
        
        if q_matrix_path:
            print('Loading existing q')
            self.q = np.load(q_matrix_path)
        else:
            self.q = np.zeros((len(self.high_level_state_codes), len(self.high_level_actions)))
        
        
    def get_high_level_action(self, player, the_map):
        '''Get a high level action for the player based on high level percepts'''
        #Create high level state from percept derived from player and map state
        high_level_state = self.get_high_level_state(player, the_map)
        state_idx = self.high_level_state_codes.index(high_level_state)
        
        #this should only be called for live players, not incapacitated players, but we'll check
        if player.is_incapacitated:
            return 'wait', high_level_state
        
        #select best action with probability proportional to utility
        action_utilities = {}
        best_hla = ''
        best_utility = 0.0
        
        #hla = high level action
        for hla in self.get_available_hlas(player, the_map, high_level_state):
            action_idx = self.high_level_actions.index(hla)
            value = self.q[state_idx, action_idx]
            action_utilities[hla] = value
            if value>best_utility:
                best_utility=value
                best_hla=hla
        
        #sometimes choose best, sometimes choose with probability
        if random.random()<0.25:
            #normalize to positives
            min_val = min(action_utilities.values())
            action_utilities = {hla:v-min_val+self.epsilon for hla,v in action_utilities.items()}
            actions = [k for k in action_utilities.keys()]
            scale = sum(action_utilities.values())
            probs = [v/scale for v in action_utilities.values()]
            idx = np.random.choice(len(actions), p=probs)
            action = actions[idx]
        else:
            action = best_hla
        
        return action, high_level_state
    
    
    def get_high_level_state(self, player, the_map):
        '''Determine which high level state applies at this time'''
        state = [0 for i in range(len(self.high_level_states))]
        
        #if player is incapacitated, that is all that matters, this saves making pointless states
        if player.is_incapacitated:
            state[self.high_level_states.index('self_incapacitated')] = 1   
            return tuple(state)
        
        if (player.team=='blue' and the_map.red_flag_in_play) or (player.team=='red' and the_map.blue_flag_in_play):
            state[self.high_level_states.index('opponents_flag_in_play')] = 1
        if (player.team=='blue' and the_map.blue_flag_in_play) or (player.team=='red' and the_map.red_flag_in_play):
            state[self.high_level_states.index('team_flag_in_play')] = 1
        if player.has_flag:
            state[self.high_level_states.index('self_has_flag')] = 1
        if player.in_enemy_territory:
            state[self.high_level_states.index('self_in_enemy_territory')] = 1
            
        #this is different in PolicyTrainer, the_map has this method
        teammate_player_info = player.get_closest_player_info_by_team(player.team)
        if teammate_player_info['has_flag']:
            state[self.high_level_states.index('nearest_teammate_has_flag')] = 1
        if teammate_player_info['is_incapacitated']:
            state[self.high_level_states.index('nearest_teammate_incapacitated')] = 1
        
        #this is different in PolicyTrainer, the_map has this method
        opponent_player_info = player.get_closest_player_info_by_team('red' if player.team=='blue' else 'blue')
        if opponent_player_info['has_flag']:
            state[self.high_level_states.index('nearest_opponent_has_flag')] = 1
        if opponent_player_info['is_incapacitated']:
            state[self.high_level_states.index('nearest_opponent_incapacitated')] = 1
            
        return tuple(state)
            
        
    def get_available_hlas(self, player, the_map, high_level_state):
        '''Determine possible high level actions given the high level state'''
        #almost all hlas are possible except the two dependent on the flag being carried
        if player.is_incapacitated:
            return ['wait']
        
        hlas = self.high_level_actions.copy()
        hlas.remove('go_opponent_flag_carrier')
        hlas.remove('gaurd_teammate_flag_carrier')
        hlas.remove('go_nearest_incapacitated_teammate')
        
        if (player.team=='blue' and the_map.blue_flag_in_play) or (player.team=='red' and not the_map.red_flag_in_play):
            hlas.append('go_opponent_flag_carrier')
        if not player.has_flag and (player.team=='blue' and the_map.red_flag_in_play) or (player.team=='red' and not the_map.blue_flag_in_play):
            hlas.append('gaurd_teammate_flag_carrier')
        if high_level_state[self.high_level_states.index('nearest_teammate_incapacitated')]==1:
            hlas.append('go_nearest_incapacitated_teammate')
            
        return hlas

In [7]:
#MAKE SURE any changes here are reflected in app/player.py

class Player():
    def __init__(self, x, y, idx, team, the_map, config):
        '''
        x,y - the player/agent sprite starting location on the map
        team - blue or red
        the_map - a reference to the global map with speeds, flag locations, player locations
        config - configurable settings
        '''
        self.config = config
        
        #needed for preventing sprite from overlapping a 0 speed location
        self.half_size = config.terrain_tile_size//2
        
        #w,a,d,s are the keyboard directions for up, left, right, down
        self.actions = ['w', 'a', 'd', 's']
        
        #for debugging
        self.speed_terr = {v:k for k,v in self.config.terrain_speeds.items()}
        
        #configured values
        
        #TODO - implement variable default speeds so some players are naturally faster
        #self.max_speed = self.config.player_max_speed
        
        #TODO implement energy countdown to promote teamwork through revivals
        #self.max_energy = self.config.player_max_energy
        #self.min_energy = self.config.player_min_energy
        #self.energy = self.config.player_max_energy
        
        self.player_idx = idx
        self.team = team
        self.the_map = the_map
        self.sprite = None
        
        #current status
        self.speed = self.config.player_max_speed
        self.has_flag = False
        self.is_incapacitated = False
        self.incapacitated_countdown = 0
        self.in_enemy_territory = False
        self.in_flag_area = False
        
        self.x = x
        self.y = y
        self.tile_col, self.tile_row = self.the_map.xy_to_cr(x, y)
        self.blocked_countdown = 0
        
    
    def update(self, frame):
        #get movement action based on map, players, flag percepts
        action = self.get_action()
        
        #center of sprite, before moving the sprite test to see if the position is legal
        new_x, new_y = self.x, self.y
        
        #for checking sprite overlap of 0 speed area, only one side needs to be checked
        delta = (0,0)
        
        #regardless of legality of move, animate the sprite
        
        #move right
        if action=='d':
            pyg.changeSpriteImage(self.sprite, 0*8+frame)    
            new_x += self.speed
            #new_x is center of sprite, adding half size in movement direction prevents overlap 0 speed tile
            #delta = (self.half_size, 0)
            
        #move down
        elif action=='s':
            pyg.changeSpriteImage(self.sprite, 1*8+frame)    
            new_y += self.speed
            #delta = (0, self.half_size)
            
        #move left
        elif action=='a':
            pyg.changeSpriteImage(self.sprite, 2*8+frame)    
            new_x -= self.speed
            #delta = (-self.half_size, 0)
            
        #move up
        elif action=='w':
            pyg.changeSpriteImage(self.sprite, 3*8+frame)
            new_y -= self.speed
            #delta = (0, -self.half_size)
            
        #stay still
        else:
            pyg.changeSpriteImage(self.sprite, 1 * 8 + 5)
            
        #only allow movement if not incapacitated
        if not self.is_incapacitated:
            #determine which tile/grid of the map would this put the player in    
            tile_col, tile_row = self.the_map.xy_to_cr(new_x + delta[0], new_y + delta[1])

            speed = self.the_map.get_speed(tile_col, tile_row)

            self.in_enemy_territory = self.the_map.in_enemy_territory(self.team, tile_col)
            self.in_flag_area = self.the_map.in_flag_area(self.team, tile_col, tile_row)

            if speed:
                self.blocked_countdown = 0
                
                #for debugging
                if self.config.verbose and speed != self.speed:
                    print('%s player %d moved from %s to %s, speed=%d, yx=(%d, %d), rc=(%d, %d)' % (self.team, self.player_idx, 
                           self.speed_terr[self.speed], self.speed_terr[speed], speed, new_y, new_x, tile_row, tile_col))

                self.speed = speed
                self.x = new_x
                self.y = new_y
                
                #update global map with change in position
                #self.the_map.agent_info[self.team][self.player_idx]['xy'] = (new_x, new_y)

                #pyg.moveSprite(self.sprite, self.x, self.y)
            else:
                #going at the terrain speed would land player on 0, go less than full speed to reach limit of movement
                if delta[0]<0:
                    new_x = ((tile_col+1) * self.config.terrain_tile_size) #+ self.half_size
                elif delta[0]>0:
                    new_x = ((tile_col-1) * self.config.terrain_tile_size) #+ self.half_size
                elif delta[1]<0:
                    new_y = ((tile_row+1) * self.config.terrain_tile_size) #+ self.half_size
                elif delta[1]>0:
                    new_y = ((tile_row-1) * self.config.terrain_tile_size) #+ self.half_size

                self.blocked_countdown = 20
                
            #update global map with change in position
            self.the_map.agent_info[self.team][self.player_idx]['xy'] = (new_x, new_y)

            pyg.moveSprite(self.sprite, self.x, self.y)
            
        
    def get_action(self):
        pass
    
    
    def update_sprite(self, sprite):
        pyg.hideSprite(self.sprite)
        self.sprite = sprite
        pyg.moveSprite(self.sprite, self.x, self.y, centre=True)
        pyg.showSprite(self.sprite)
        
        
    def get_direction_to_xy(self, xy):
        x,y = xy
        delta_x, delta_y = x - self.x, y - self.y
        if abs(delta_x)>abs(delta_y):
            return 'a' if delta_x<0 else 'd'
        else:
            return 'w' if delta_y<0 else 's'
        
        
    def get_direction_away_from(self, xy):
        x,y = xy
        delta_x, delta_y = self.x - x, self.y - y
        if abs(delta_x)<abs(delta_y):
            return 'a' if delta_x<0 else 'd'
        else:
            return 'w' if delta_y<0 else 's'
        
        
    def go_between(self, xy1, xy2):
        x1,y1 = xy1
        x2,y2 = xy2
        midx, midy = x1 + ((x2 - x1)/2), y1 + ((y2 - y1)/2)
        return self.get_direction_to_xy((midx, midy))

        
    def get_closest_player_info_by_team(self, team):
        best_dist=float('inf')
        best_info = {}
        
        for idx, info in self.the_map.agent_info[team].items():
            if idx==self.player_idx:
                continue
                
            x, y = info['xy']
            dist = np.sqrt((self.x - x)**2 + (self.y - y)**2)
            if dist<best_dist:
                best_info = info
                
        return best_info
    
    
    def get_closest_player_info_to_xy_by_team(self, xy, team):
        best_dist=float('inf')
        best_info = {}
        x1, y1 = xy
        for idx, info in self.the_map.agent_info[team].items():
            if idx==self.player_idx:
                continue
                
            x2, y2 = info['xy']
            dist = np.sqrt((x1 - x2)**2 + (y1 - y2)**2)
            if dist<best_dist:
                best_info = info
                
        return best_info
    
    
    def get_closest_incapacitated_player_info_by_team(self, team):
        best_dist=float('inf')
        best_info = {}
        
        for idx, info in self.the_map.agent_info[team].items():
            if idx==self.player_idx:
                continue
                
            if not info['is_incapacitated']:
                continue
                
            x, y = info['xy']
            dist = np.sqrt((self.x - x)**2 + (self.y - y)**2)
            if dist<best_dist:
                best_info = info
                
        return best_info
    
    
    
class HumanPlayer(Player):
    def __init__(self, x, y, idx, team, the_map, config):
        super().__init__(x, y, idx, team, the_map, config)
        #load sprites
        #there are flag holding, nonflag holding, and incapacitated sprites for blue player (human), blue agent, red agent
        #current sprite may change to flag holding or incapacitated sprite
        self.default_sprite = pyg.makeSprite(self.config.blue_player_sprite_path, 32)
        self.holding_flag_sprite = pyg.makeSprite(self.config.blue_player_with_flag_sprite_path, 32)
        self.incapacitated_sprite = pyg.makeSprite(self.config.blue_player_incapacitated_sprite_path, 32)
                                                   
        self.sprite = self.default_sprite
        
        pyg.moveSprite(self.sprite, x, y, centre=True)
        pyg.showSprite(self.sprite)
        
        
    def get_action(self):
        new_x, new_y = self.x, self.y
        
        #move right
        if pyg.keyPressed("d"):
            return 'd'
            
        #move down
        elif pyg.keyPressed("s"):
            return 's'
            
        #move left
        elif pyg.keyPressed("a"):
            return 'a'
            
        #move up
        elif pyg.keyPressed("w"):
            return 'w'
    
    
    
class AgentPlayer(Player):
    '''
    Moves around randomly
    '''
    def __init__(self, x, y, idx, team, the_map, config):
        super().__init__(x, y, idx, team, the_map, config)
        self.prev_dir = 'a' if team=='red' else 'd'
        
        #load sprites
        #there are flag holding, nonflag holding, and incapacitated sprites for blue player (human), blue agent, red agent
        #current sprite may change to flag holding or incapacitated sprite
        if team=='blue':
            self.default_sprite = pyg.makeSprite(self.config.blue_agent_sprite_path, 32)
            self.holding_flag_sprite = pyg.makeSprite(self.config.blue_agent_with_flag_sprite_path, 32)
            self.incapacitated_sprite = pyg.makeSprite(self.config.blue_agent_incapacitated_sprite_path, 32)
        else:
            self.default_sprite = pyg.makeSprite(self.config.red_agent_sprite_path, 32)
            self.holding_flag_sprite = pyg.makeSprite(self.config.red_agent_with_flag_sprite_path, 32)
            self.incapacitated_sprite = pyg.makeSprite(self.config.red_agent_incapacitated_sprite_path, 32)
            
        self.sprite = self.default_sprite
        
        pyg.moveSprite(self.sprite, x, y, centre=True)
        pyg.showSprite(self.sprite)
        
        
    def get_action(self):
        if random.randint(1,20) == 1:
            self.prev_dir = random.choice(['a','w','s','d'])
            
        return self.prev_dir
    
    
    
class GreedyGoalAgentPlayer(AgentPlayer):
    '''
    A reflex agent following the rules:
    - If the flag is not in play head directly for it 
    - If player has the flag head directly for flag area
    - If opponent has flag head directly toward them
    '''
    def __init__(self, x, y, idx, team, the_map, config):
        super().__init__(x, y, idx, team, the_map, config)
        
        
    def get_action(self):
        action = ''
        
        #if blocked going direct way, try going a random direction for a while
        if self.blocked_countdown:
            if self.blocked_countdown==20:
                dirs = ['a','w','s','d']
                dirs.remove(self.prev_dir)
                self.prev_dir = random.choice(dirs)
                
            self.blocked_countdown -= 1
            
            if random.randint(1,10) == 1:
                self.prev_dir = random.choice(['a','w','s','d'])
                
            action = self.prev_dir
            if self.config.verbose:
                print('%s player %d is blocked, trying different way: %s' % (self.team, self.player_idx, action))
                
            return action
        
        
        #head to flag home area
        if self.has_flag:
            if self.team=='blue':
                action = self.get_direction_to_xy(self.the_map.blue_flag_xy)
            else:
                action = self.get_direction_to_xy(self.the_map.red_flag_xy)
            if self.config.verbose:
                print('%s player %d heading to flag area: %s' % (self.team, self.player_idx, action))
            return action
        
                
        #the flag is being run by an opponent, try to tag them
        if self.the_map.blue_flag_in_play and self.team=='blue':
            agent_info = self.the_map.agent_info['red']
            for idx, info in agent_info.items():
                if info['has_flag']:
                    action = self.get_direction_to_xy(info['xy'])
                    if self.config.verbose:
                        print('%s player %d heading to tag opponent %d: %s' % (self.team, self.player_idx, idx, action))
            return action
        if self.the_map.red_flag_in_play and self.team=='red':
            agent_info = self.the_map.agent_info['blue']
            for idx, info in agent_info.items():
                if info['has_flag']:
                    action = self.get_direction_to_xy(info['xy'])
                    if self.config.verbose:
                        print('%s player %d heading to tag opponent %d: %s' % (self.team, self.player_idx, idx, action))
            return action
                
        
        #head to flag
        if self.team=='blue':# and not self.the_map.red_flag_in_play:
            action = self.get_direction_to_xy(self.the_map.red_flag_xy)
            if self.config.verbose:
                print('%s player %d heading to flag: %s' % (self.team, self.player_idx, action))
        elif self.team=='red':# and not self.the_map.blue_flag_in_play:
            action = self.get_direction_to_xy(self.the_map.blue_flag_xy)
            if self.config.verbose:
                print('%s player %d heading to flag: %s' % (self.team, self.player_idx, action))
        
           
        return action
    
    

class PolicyAgentPlayer(AgentPlayer):
    '''
    This Agent follows a learned high-level policy to determine its actions
    '''
    def __init__(self, x, y, idx, team, the_map, config, policy):
        super().__init__(x, y, idx, team, the_map, config)
        self.policy = policy
        
        
    def get_action(self):
        action = ''
        high_level_action, high_level_state = self.policy.get_high_level_action(self, self.the_map)
        
        #translate HLA into a direction to move in
        action = self.__hla_to_direction(high_level_action)
        
        hls=[]
        for i in range(len(high_level_state)):
            if high_level_state[i]==1:
                hls.append(self.policy.high_level_states[i])
        print('%s agent %d, hla: %s, action: %s\nhls: %s\n%s' % (self.team, self.player_idx, high_level_action, 
                                                             action, high_level_state, ', '.join(hls)))
        
        return action
    
    
    def __hla_to_direction(self, hla):
        action = ''
        opponent_team = 'red' if self.team=='blue' else 'blue'
        
        if hla=='random':
            if random.randint(1,20) == 1:
                self.prev_dir = random.choice(['a','w','s','d'])
            action = self.prev_dir
        elif hla=='go_opponent_flag':
            xy = self.the_map.red_flag_xy if self.team=='blue' else self.the_map.blue_flag_xy
            action = self.get_direction_to_xy(xy)
        elif hla=='go_team_flag_area':
            xy = self.the_map.blue_flag_xy if self.team=='blue' else self.the_map.red_flag_xy
            action = self.get_direction_to_xy(xy)
        elif hla=='go_opponent_flag_carrier':
            for info in self.the_map.agent_info[opponent_team].values():
                if info['has_flag']:
                    action = self.get_direction_to_xy(info['xy'])
        elif hla=='go_nearest_opponent':
            info = self.get_closest_player_info_by_team(opponent_team)
            action = self.get_direction_to_xy(info['xy'])
        elif hla=='go_nearest_teammate':
            info = self.get_closest_player_info_by_team(self.team)
            action = self.get_direction_to_xy(info['xy'])
        elif hla=='go_nearest_incapacitated_teammate':
            info = self.get_closest_incapacitated_player_info_by_team(self.team)
            action = self.get_direction_to_xy(info['xy'])
        elif hla=='gaurd_nearest_teammate':
            info = self.get_closest_player_info_by_team(self.team)
            opp_info = self.get_closest_player_info_to_xy_by_team(info['xy'], opponent_team)
            action = self.go_between(info['xy'], opp_info['xy'])
        elif hla=='gaurd_teammate_flag_carrier':
            for info in self.the_map.agent_info[self.team].values():
                if info['has_flag']:
                    action = self.get_direction_to_xy(info['xy'])
        elif hla=='gaurd_team_flag_area':
            xy1 = self.the_map.blue_flag_xy if self.team=='blue' else self.the_map.red_flag_xy
            info = self.get_closest_player_info_to_xy_by_team(xy1, opponent_team)
            action = self.go_between(xy1, info['xy'])
        elif hla=='guard_opponent_flag_area':
            xy1 = self.the_map.red_flag_xy if self.team=='blue' else self.the_map.blue_flag_xy
            info = self.get_closest_player_info_to_xy_by_team(xy1, opponent_team)
            action = self.go_between(xy1, info['xy'])
        elif hla=='run_away_from_nearest_opponent':
            info = self.get_closest_player_info_by_team(opponent_team)
            action = self.get_direction_away_from(info['xy'])
        elif hla=='run_away_from_opponents_centroid':
            xs, ys = [],[]
            for info in self.the_map.agent_info[opponent_team].values():
                x,y = info['xy']
                xs.append(x)
                ys.append(y)
            mean_xy = (sum(xs)/len(xs), sum(ys)/len(ys))
            action = self.get_direction_away_from(mean_xy)
            
        return action
    
    
class AStarAgentPlayer(AgentPlayer):
    '''
    This Agent follows a learned high-level policy to determine its actions
    '''
    def __init__(self, x, y, idx, team, the_map, config):
        super().__init__(x, y, idx, team, the_map, config)
        
        self.goals = ['opponent_flag', 'team_flag_area', 'opponent_flag_area'] #other...
        self.current_goal = 'opponent_flag'
        goal_location = the_map.red_flag_xy if team=='blue' else the_map.blue_flag_xy
        self.current_path = self.__a_star((x, y), goal_location)
        
        
    def get_action(self):
        action = ''
        
        if self.is_incapacitated:
            return ''
        
        #recalc A* due to goal change?
        if self.has_flag:
            if not self.current_goal=='team_flag_area':
                self.current_goal = 'team_flag_area'
                goal_location = self.the_map.blue_flag_xy if team=='blue' else self.the_map.red_flag_xy
                self.current_path = self.__a_star((self.x, self.y), goal_location)
        
        #go after other team flag carrier or flag area
        elif (self.team=='blue' and self.the_map.blue_flag_in_play) or (self.team=='red' and self.the_map.red_flag_in_play):
            #if far from flag carrier go towards their flag location, else go directly toward them (no A*)
            
            if not self.current_goal=='opponent_flag_area':
                self.current_goal = 'opponent_flag_area'
                goal_location = self.the_map.red_flag_xy if team=='blue' else self.the_map.blue_flag_xy
                self.current_path = self.__a_star((self.x, self.y), goal_location)
        #etc
        
        if self.current_path:
            action = self.current_path.pop()
        else:
            #default action - head directly toward opponent flag area 
            goal_location = self.the_map.red_flag_xy if team=='blue' else self.the_map.blue_flag_xy
            action = self.get_direction_to_xy(goal_location)
        
        return action
                
        
    def __a_star(self, xy1, xy2):
        path = []
        # A*
        return path

In [9]:
#MAKE SURE any changes here are reflected in app/capture_the_flag.py

class CaptureTheFlag():
    def __init__(self, config, new_map_seed=0):
        '''
        Load background, icon, music, sounds, create flag sprites and areas, create players
        '''
        self.config = config
        self.policy = Policy('q.npy')
        
        #sounds
        self.grabbed_flag_sound = pyg.makeSound(self.config.grabbed_flag_sound)
        self.incapacitated_sound = pyg.makeSound(self.config.incapacitated_sound)
        self.dropped_flag_sound = pyg.makeSound(self.config.dropped_flag_sound)
        self.tagged_sound = pyg.makeSound(self.config.tagged_sound)
        self.tagged_flag_carrier_sound = pyg.makeSound(self.config.tagged_flag_carrier_sound)
        self.revived_sound = pyg.makeSound(self.config.revived_sound)
        self.victory_sound = pyg.makeSound(self.config.victory_sound)
        
        #for drawing the divide and knowing when someone is off sides
        self.screen_divide_x = (self.config.screen_width//2)
        
        #stores location of terrain speeds including 0 (not allowed areas), flag location, player locations
        self.the_map = TheMap(self.config, new_map_seed)
        
        
        #set screen, title, icon, background, music
        self.__setup(self.the_map.map_path)

        
        #make flags
        self.blue_flag_sprite, self.red_flag_sprite = self.__make_flags()
        
        
        #make players with reference to map, and with their locations added to the map
        allowed_blue_sprite_init_tiles = self.__get_allowed_sprite_init_tiles('blue')
    
        #create user player on blue team
        self.user_player = self.__make_player(allowed_blue_sprite_init_tiles, idx=0, team='blue', agent=False)
        
        #create other blue players
        self.blue_players = [self.user_player]
        for i in range(self.config.blue_team_size - 1):
            blue_player = self.__make_player(allowed_blue_sprite_init_tiles, idx=i+1, team='blue')
            self.blue_players.append(blue_player)

            
        #get red side non-lake non border tile locations
        allowed_red_sprite_init_tiles = self.__get_allowed_sprite_init_tiles('red')
        
        #create red players
        self.red_players = []
        for i in range(self.config.red_team_size):
            red_player = self.__make_player(allowed_red_sprite_init_tiles, idx=i, team='red')
            self.red_players.append(red_player)
            
        #all players
        self.players = self.blue_players + self.red_players
        
    
    def run(self):
        nextFrame = pyg.clock()
        frame = 0
        
        pyg.pause(5000)
        
        while True:
            #quit?
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    pygame.quit()
                    sys.exit()
                    
            # We only animate our character every 80ms.
            if pyg.clock() > nextFrame:
                self.__draw_divide()
                self.__draw_flag_areas()
                # There are 8 frames of animation in each direction so the modulus 8 allows it to loop                        
                frame = (frame+1)%8                         
                nextFrame += 80 

            #execute each players action
            for player in self.players:
                #print('updating player %d' % player.player_idx)
                player.update(frame)
            
            #game logic here
            state = self.__get_state()
            
            #handle states
            if state['blue_wins']:
                pygame.mixer.music.fadeout(1000)
                pyg.pause(1000)
                win_label = pyg.makeLabel('BLUE WINS!!!', 80, self.config.screen_width//2-200, self.config.screen_height//2-40, 
                              fontColour='blue', font='Arial', background="black")
                pyg.showLabel(win_label)
                self.end_game_music = pyg.makeMusic(self.config.end_game_music)
                pyg.playMusic()
                pyg.pause(5000)
                pygame.mixer.music.fadeout(3000)
                quit_label = pyg.makeLabel('Press Esc', 24, self.config.screen_width//2-30, self.config.screen_height//2+120, 
                              fontColour='blue', font='Arial', background="black")
                pyg.showLabel(quit_label)
                pyg.pause(3000)
                break
            elif state['red_wins']:
                pygame.mixer.music.fadeout(1000)
                pyg.pause(1000)
                win_label = pyg.makeLabel('RED WINS!!!', 80, self.config.screen_width//2-200, self.config.screen_height//2-40, 
                              fontColour='red', font='Arial', background="black")
                pyg.showLabel(win_label)
                self.end_game_music = pyg.makeMusic(self.config.end_game_music)
                pyg.playMusic()
                pyg.pause(5000)
                pygame.mixer.music.fadeout(3000)
                quit_label = pyg.makeLabel('Press Esc', 24, self.config.screen_width//2-30, self.config.screen_height//2+120, 
                              fontColour='red', font='Arial', background="black")
                pyg.showLabel(quit_label)
                pyg.pause(3000)
                break
           

            pyg.tick(10)

        pyg.endWait()
        
        
    def __get_state(self):
        '''Game logic here.'''
        state = {'blue_wins':False, 'red_wins':False}
        
        #all states can be determined by what the players are touching and where they are
        for player in self.players:
            
            #can player do anything?
            if player.is_incapacitated:
                player.incapacitated_countdown -= 1
                
                #is countdown over?
                if player.incapacitated_countdown<=0:
                    print('%s player is no longer incapacitated' % player.team)
                    self.__handle_revival(player)
                #player revived - TODO use sprite group
                elif self.__tagged_by_team_member(player):
                    print('%s player revived' % player.team)
                    self.__handle_revival(player)
                continue
                
                
            #check for win condition
            if player.has_flag and player.in_flag_area:
                if player.team=='blue':
                    print('blue wins')
                    state['blue_wins']=True
                else:
                    print('red wins')
                    state['red_wins']=True
                break
                
            
            #this player isn't touching anything
            if not pyg.allTouching(player.sprite):
                continue
                
                
            #flag grabbed?
            if not self.the_map.blue_flag_in_play and player.team=='red' and pyg.touching(player.sprite, self.blue_flag_sprite):
                print('%s player %d touched blue flag' % (player.team, player.player_idx))
                self.__handle_flag_grabbed(player, self.blue_flag_sprite)
                continue
            elif not self.the_map.red_flag_in_play and player.team=='blue' and pyg.touching(player.sprite, self.red_flag_sprite):
                print('%s player %d touched blue flag' % (player.team, player.player_idx))
                self.__handle_flag_grabbed(player, self.red_flag_sprite)
                continue
        
        
            #check for player tagged - TODO make team player sprite Groups instead of looping
            if player.in_enemy_territory or player.has_flag:
                if player.team=='blue':
                    if any([pyg.touching(player.sprite, red_player.sprite)!=None for red_player in self.red_players]):
                        print('blue player tagged')
                        self.__handle_tag(player)
                        continue
                elif player.team=='red':
                    if any([pyg.touching(player.sprite, blue_player.sprite)!=None for blue_player in self.blue_players]):
                        print('red player tagged')
                        self.__handle_tag(player)
                        continue
        
        return state
    
    
    #state actions
    
    def __handle_flag_grabbed(self, player, flag_sprite):
        pyg.playSound(self.grabbed_flag_sound)
        pyg.hideSprite(flag_sprite)
        player.has_flag = True
        player.update_sprite(player.holding_flag_sprite)
        
        #update the global map
        if player.team=='blue':
            self.the_map.red_flag_in_play = True
            self.the_map.agent_info['blue'][player.player_idx]['has_flag'] = True
        else:
            self.the_map.blue_flag_in_play = True
            self.the_map.agent_info['red'][player.player_idx]['has_flag'] = True
        
        
    def __handle_tag(self, player):
        if player.has_flag:
            pyg.playSound(self.tagged_flag_carrier_sound)
            player.has_flag = False
            
            #return flag (or drop it near them? then it will never go backwards)
            if player.team=='blue':
                pyg.moveSprite(self.red_flag_sprite, x=self.red_flag_x, y=self.red_flag_y, centre=True)
                pyg.showSprite(self.red_flag_sprite)
                
                #update the global map
                self.the_map.red_flag_in_play = False
                self.the_map.agent_info['blue'][player.player_idx]['has_flag'] = False
                self.the_map.agent_info['blue'][player.player_idx]['is_incapacitated'] = True
            else:
                pyg.moveSprite(self.blue_flag_sprite, x=self.blue_flag_x, y=self.blue_flag_y, centre=True)
                pyg.showSprite(self.blue_flag_sprite)
                
                #update the global map
                self.the_map.blue_flag_in_play = False
                self.the_map.agent_info['red'][player.player_idx]['has_flag'] = False
                self.the_map.agent_info['red'][player.player_idx]['is_incapacitated'] = True
        else:
            pyg.playSound(self.tagged_sound)
            
        pyg.playSound(self.incapacitated_sound)
        
        player.is_incapacitated = True
        #player.energy = 0
        player.update_sprite(player.incapacitated_sprite)
        player.incapacitated_countdown = 60
        
            
    def __tagged_by_team_member(self, player):
        if player.team=='blue':
            for blue_player in self.blue_players:
                #can't revive yourself!!!
                if not player==blue_player and pyg.touching(player.sprite, blue_player.sprite):
                    return True
        else:
            for red_player in self.red_players:
                if not player==red_player and pyg.touching(player.sprite, red_player.sprite):
                    return True
                
        return False
                                                            
            
    def __handle_revival(self, player):
        pyg.playSound(self.revived_sound)
        player.is_incapacitated = False
        #player.energy = self.config.player_max_energy
        player.update_sprite(player.default_sprite)
        player.incapacitated_countdown = 0
        
        #update global map
        if player.team=='blue':
            self.the_map.agent_info['blue'][player.player_idx]['is_incapacitated'] = False
        else:
            self.the_map.agent_info['red'][player.player_idx]['is_incapacitated'] = False
        
        
    def __setup(self, map_path):
        pyg.screenSize(self.config.screen_width, self.config.screen_height)
        pyg.setIcon(self.config.game_icon_image_path)
        pyg.setWindowTitle(self.config.game_name)
        
        #load the map
        pyg.setBackgroundImage(map_path)
        
        #draw dividing line
        self.__draw_divide()
        
        #music
        pyg.makeMusic(self.config.default_game_music)
        pyg.playMusic(loops=-1)
        #TODO
        #self.single_flag_runner_music = pyg.makeMusic(self.config)
        #self.double_flag_runner_music = pyg.makeMusic(self.config)
        
        
    def __draw_divide(self):
        pyg.drawLine(self.screen_divide_x-3, self.config.map_border_size,
                     self.screen_divide_x-3, self.config.screen_height - self.config.map_border_size, 
                     'blue', linewidth=3)
        
        pyg.drawLine(self.screen_divide_x, self.config.map_border_size, 
                     self.screen_divide_x, self.config.screen_height - self.config.map_border_size, 
                     'red', linewidth=3)
    
    
    def __make_flags(self):
        self.blue_flag_x, self.blue_flag_y = self.__get_flag_position(
            self.the_map, 'blue')
        self.red_flag_x, self.red_flag_y = self.__get_flag_position(
            self.the_map, 'red')
        
        self.__draw_flag_areas()
        
        blue_flag_sprite = pyg.makeSprite(self.config.blue_flag_sprite_path)
        pyg.moveSprite(blue_flag_sprite, x=self.blue_flag_x, y=self.blue_flag_y, centre=True)
        pyg.showSprite(blue_flag_sprite)
        
        red_flag_sprite = pyg.makeSprite(self.config.red_flag_sprite_path)
        pyg.moveSprite(red_flag_sprite, x=self.red_flag_x, y=self.red_flag_y, centre=True)
        pyg.showSprite(red_flag_sprite)
        
        #update the map object with their locations
        self.the_map.set_flag_location('blue', self.blue_flag_x, self.blue_flag_y)
        self.the_map.set_flag_location('red', self.red_flag_x, self.red_flag_y)
        
        return blue_flag_sprite, red_flag_sprite
        
        
    def __get_flag_position(self, the_map, team):
        #choose an available row along a column a set distance from the border
        pad = 20
        
        if team=='blue':
            x = the_map.border_size + self.config.flag_area_size//2 + pad
        else:
            x = self.config.screen_width - the_map.border_size - pad - self.config.flag_area_size//2
            
        flag_tile_c = x//the_map.tile_size
        
        col_speeds = the_map.tile_speeds[:, flag_tile_c]
        idx = np.where(col_speeds > 0)[0].tolist()
        
        #trim top and bottow 2 options to avoid map area going off the screen
        flag_tile_r = random.choice(idx[2:-2])
        y = flag_tile_r * the_map.tile_size
        
        return x, y

    
    def __draw_flag_areas(self):
        pyg.drawEllipse(self.blue_flag_x, self.blue_flag_y, self.config.flag_area_size, self.config.flag_area_size, 
                        colour='blue', linewidth=3)
        
        pyg.drawEllipse(self.red_flag_x, self.red_flag_y, self.config.flag_area_size, self.config.flag_area_size, 
                        colour='red', linewidth=3)
        
        
    def __get_allowed_sprite_init_tiles(self, team):
        side = self.the_map.tile_speeds.shape[1]//3
        if team=='blue':
            idx = np.where(self.the_map.tile_speeds[:,:side]>0)
        else:
            idx = np.where(self.the_map.tile_speeds[:,(side*2):]>0)
            idx = (idx[0], idx[1] + np.ones_like(idx[1]) * (side*2))
            
        allowed_init_tiles = list(zip(idx[0].tolist(), idx[1].tolist()))
        
        random.shuffle(allowed_init_tiles)
        
        return allowed_init_tiles


    def __make_player(self, allowed_sprite_init_tiles, idx=-1, team='', agent=True):
        player_r_idx, player_c_idx = allowed_sprite_init_tiles.pop()

        player_x_pos = (player_c_idx * self.config.terrain_tile_size) + self.config.terrain_tile_size//2
        player_y_pos = (player_r_idx * self.config.terrain_tile_size) + self.config.terrain_tile_size//2
        
        if self.config.verbose:
            print('%s %s on r=%s, c=%d, y=%d, x=%d' % (team, 'agent' if agent else 'player', 
                                                   player_r_idx, player_c_idx, player_y_pos, player_x_pos))

        if not agent: #this will be a different sprite
            player = HumanPlayer(player_x_pos, player_y_pos, idx, team, self.the_map, self.config)
        else:
            #player = AgentPlayer(player_x_pos, player_y_pos, idx, team, self.the_map, self.config)
            if team=='blue':
                #player = PolicyAgentPlayer(player_x_pos, player_y_pos, idx, team, self.the_map, self.config, self.policy)
                player = GreedyGoalAgentPlayer(player_x_pos, player_y_pos, idx, team, self.the_map, self.config)
            else:
                player = GreedyGoalAgentPlayer(player_x_pos, player_y_pos, idx, team, self.the_map, self.config)
            
        #set info in map
        player_info = {'xy':(player_x_pos, player_y_pos), 
                       'has_flag':False,
                       'is_incapacitated':False, 
                       'in_enemy_territory':False}
        self.the_map.agent_info[team][idx] = player_info

        return player

In [10]:
%tb

config = Config(verbose=False)
#config.map_border_size = 10
#config.terrain_tile_size = 10
#config.map_default_path = '../app/resources/maps/map_870_670.png'
#config.map_default_speed_array = '../app/resources/maps/map_speed_870_670.npy'

game = CaptureTheFlag(config, new_map_seed=7718)
game.run()

No traceback available to show.


Loading existing q
saving perlin_oct4_pers0.5_grid50x50.png
(32, 42)
saving map and map speeds array ../app/resources/maps/map_840_640.png
blue player 3 touched blue flag
red player tagged
blue player tagged
blue player tagged
blue player revived
blue player revived
blue player 1 touched blue flag
blue player tagged
blue player tagged
blue player tagged
red player is no longer incapacitated
red player tagged
blue player is no longer incapacitated
blue player revived
blue player revived
blue player 1 touched blue flag
red player is no longer incapacitated
blue player tagged
red player tagged
blue player 0 touched blue flag
red player tagged
blue player is no longer incapacitated
red player is no longer incapacitated
red player tagged
red player is no longer incapacitated
red player is no longer incapacitated
red player tagged
red player is no longer incapacitated
red player tagged
blue player tagged
blue player 3 touched blue flag
blue player tagged
red player is no longer incapacitated

SystemExit: 

c:\ml\env\lib\site-packages\IPython\core\interactiveshell.py:3426: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [4]:
%tb

config = Config(verbose=False)

config.map_border_size = 10
config.terrain_tile_size = 10
config.map_default_path = '../app/resources/maps/map_870_670.png'
config.map_default_speed_array = '../app/resources/maps/map_speed_870_670.npy'

game = CaptureTheFlag(config, new_map_seed=0)
game.run()

No traceback available to show.


Using default map
blue flag is on 9, 8, in area [(5, 4), (5, 5), (5, 6), (5, 7), (5, 8), (5, 9), (5, 10), (5, 11), (5, 12), (6, 4), (6, 5), (6, 6), (6, 7), (6, 8), (6, 9), (6, 10), (6, 11), (6, 12), (7, 4), (7, 5), (7, 6), (7, 7), (7, 8), (7, 9), (7, 10), (7, 11), (7, 12), (8, 4), (8, 5), (8, 6), (8, 7), (8, 8), (8, 9), (8, 10), (8, 11), (8, 12), (9, 4), (9, 5), (9, 6), (9, 7), (9, 8), (9, 9), (9, 10), (9, 11), (9, 12), (10, 4), (10, 5), (10, 6), (10, 7), (10, 8), (10, 9), (10, 10), (10, 11), (10, 12), (11, 4), (11, 5), (11, 6), (11, 7), (11, 8), (11, 9), (11, 10), (11, 11), (11, 12), (12, 4), (12, 5), (12, 6), (12, 7), (12, 8), (12, 9), (12, 10), (12, 11), (12, 12), (13, 4), (13, 5), (13, 6), (13, 7), (13, 8), (13, 9), (13, 10), (13, 11), (13, 12)]
red flag is on 34, 76, in area [(30, 72), (30, 73), (30, 74), (30, 75), (30, 76), (30, 77), (30, 78), (30, 79), (30, 80), (31, 72), (31, 73), (31, 74), (31, 75), (31, 76), (31, 77), (31, 78), (31, 79), (31, 80), (32, 72), (32, 73), (32, 74)

blue player 2 heading to flag: d
blue player 2 moved from plain to hill, speed=4, yx=(625, 260), rc=(62, 26)
blue player 3 heading to flag: d
blue player 4 heading to flag: d
red player 0 heading to flag: a
red player 1 heading to flag: a
red player 1 moved from plain to swamp, speed=7, yx=(315, 573), rc=(31, 56)
red player 2 heading to flag: a
red player 3 heading to flag: a
red player 4 heading to flag: a
blue player 1 heading to flag: d
blue player 2 heading to flag: d
blue player 3 heading to flag: d
blue player 4 heading to flag: d
red player 0 heading to flag: a
red player 1 heading to flag: a
red player 2 heading to flag: a
red player 3 heading to flag: a
red player 4 heading to flag: a
blue player 1 heading to flag: d
blue player 2 heading to flag: d
blue player 3 heading to flag: d
blue player 4 heading to flag: d
red player 0 heading to flag: a
red player 1 heading to flag: a
red player 1 moved from swamp to plain, speed=10, yx=(315, 559), rc=(31, 55)
red player 2 heading to 

blue player 2 heading to flag: d
blue player 2 moved from hill to plain, speed=10, yx=(625, 346), rc=(62, 35)
blue player 3 heading to flag: d
blue player 4 heading to flag: d
red player 0 heading to flag: a
red player 1 heading to flag: a
red player 2 heading to flag: a
red player 2 moved from plain to swamp, speed=7, yx=(265, 593), rc=(26, 58)
red player 3 heading to flag: a
red player 4 heading to flag: a
red player 4 moved from swamp to plain, speed=10, yx=(325, 538), rc=(32, 53)
blue player 1 heading to flag: d
blue player 2 heading to flag: d
blue player 3 heading to flag: d
blue player 4 heading to flag: d
red player 0 heading to flag: a
red player 1 heading to flag: a
red player 2 heading to flag: a
red player 3 is blocked, trying different way: d
red player 3 moved from swamp to plain, speed=10, yx=(22, 565), rc=(2, 57)
red player 4 heading to flag: a
blue player 1 heading to flag: d
blue player 2 heading to flag: d
blue player 3 heading to flag: d
blue player 4 heading to fla

blue player 2 heading to flag: d
blue player 2 moved from swamp to plain, speed=10, yx=(594, 511), rc=(59, 51)
blue player 3 heading to flag: s
blue player 4 heading to flag: d
red player 0 heading to flag: w
red player 1 heading to flag: a
red player 2 heading to flag: a
red player 3 is blocked, trying different way: w
red player 4 heading to flag: a
red player 4 moved from hill to plain, speed=10, yx=(325, 402), rc=(32, 39)
blue player 1 heading to flag: d
blue player 2 heading to flag: w
blue player 2 moved from plain to swamp, speed=7, yx=(584, 511), rc=(57, 51)
blue player 3 is blocked, trying different way: a
blue player 4 heading to flag: d
red player 0 heading to flag: a
red player 0 moved from swamp to plain, speed=10, yx=(353, 343), rc=(35, 33)
red player 1 heading to flag: w
red player 2 heading to flag: a
red player 3 heading to flag: a
red player 4 heading to flag: a
blue player 1 heading to flag: d
blue player 2 heading to flag: d
blue player 3 heading to flag: d
blue pla

...

red player 0 is blocked, trying different way: s
red player 1 is blocked, trying different way: w
red player 2 heading to tag opponent 0: a
red player 3 heading to tag opponent 0: a
red player 4 heading to tag opponent 0: a
red player 0 is blocked, trying different way: d
red player 1 is blocked, trying different way: s
red player 2 heading to tag opponent 0: a
red player 3 heading to tag opponent 0: a
red player 4 is blocked, trying different way: a
red player 0 heading to tag opponent 0: a
red player 1 is blocked, trying different way: w
red player 2 heading to tag opponent 0: a
red player 3 is blocked, trying different way: a
red player 4 is blocked, trying different way: d
red player 0 heading to tag opponent 0: a
red player 1 is blocked, trying different way: d
red player 2 heading to tag opponent 0: a
red player 3 is blocked, trying different way: d
red player 4 heading to tag opponent 0: a
red player 4 moved from plain to swamp, speed=7, yx=(150, 572), rc=(15, 56)
blue player 0 

red player 0 is blocked, trying different way: d
red player 1 is blocked, trying different way: w
red player 2 heading to tag opponent 0: a
red player 3 heading to tag opponent 0: a
red player 4 heading to tag opponent 0: a
red player 0 heading to tag opponent 0: a
red player 1 is blocked, trying different way: d
red player 2 heading to tag opponent 0: a
red player 3 heading to tag opponent 0: a
red player 4 is blocked, trying different way: a
red player 0 heading to tag opponent 0: a
red player 1 heading to tag opponent 0: a
red player 2 heading to tag opponent 0: a
red player 3 heading to tag opponent 0: a
red player 4 is blocked, trying different way: w
red player 0 is blocked, trying different way: s
red player 1 heading to tag opponent 0: a
red player 2 heading to tag opponent 0: w
red player 3 heading to tag opponent 0: a
red player 4 heading to tag opponent 0: a
red player 4 moved from plain to swamp, speed=7, yx=(130, 552), rc=(13, 54)
red player 0 is blocked, trying different 

red player 0 is blocked, trying different way: s
red player 1 is blocked, trying different way: d
red player 2 heading to tag opponent 0: a
red player 3 heading to tag opponent 0: a
red player 4 heading to tag opponent 0: a
red player 4 moved from plain to hill, speed=4, yx=(123, 361), rc=(12, 35)
red player 0 is blocked, trying different way: a
red player 1 heading to tag opponent 0: a
red player 2 heading to tag opponent 0: a
red player 3 heading to tag opponent 0: a
red player 4 heading to tag opponent 0: a
blue wins


SystemExit: 

c:\ml\env\lib\site-packages\IPython\core\interactiveshell.py:3426: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
